# Dask SLURMClusters

The MLeRP notebook environment uses Dask SLURMClusters to create a
middle ground that has the interactivity of a notebook backed by the
power of HPC. This notebook shows how you can use the lion service to
use a CPU based notebook session for your basic analysis and code
development. Then, when you’re ready to run tests you will use Dask to
submit your python functions to the SLURM queue.

This enables:

-   Flexibility to experiment with your dataset interactively
-   Ability to change compute requirements such as RAM, size of GPU,
    number of processes and so on… without ever leaving the notebook
    environment
-   Elastic scaling of compute
-   Efficient utilisation of the hardware
-   Releasing of resources when not in use

In [1]:
from dask_jobqueue import SLURMCluster
from distributed import Client, LocalCluster
import dask

# Point Dask to the SLURM to use as it's back end
cluster = SLURMCluster(
    memory="64g", processes=1, cores=8
)

# Scale out to 4 nodes
num_nodes = 4
cluster.scale(num_nodes)
client = Client(cluster)

Dask will now spin our jobs up in anticipation for work to the scale
that you specify.

You can check in on your jobs like you would with any other SLURM job
with `squeue`.

In [2]:
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
               383   BigCats dask-wor mhar0048 PD       0:00      1 (None)
               382   BigCats dask-wor mhar0048 PD       0:00      1 (None)
               381   BigCats dask-wor mhar0048 PD       0:00      1 (None)
               380   BigCats dask-wor mhar0048 PD       0:00      1 (None)
               373   BigCats DSKS Jup andrewpe  R    1:48:56      1 mlerp-monash-node00
               375   BigCats DSKS Jup mhar0048  R    1:15:17      1 mlerp-monash-node00

Alternatively, we can use the adapt method, which will let us scale out
as we need the compute… and scale back when we’re idle letting others
use the cluster.

We reccommend that you use the adapt method while you’re actively
developing your code so that you don’t need to worry about cleaning up
after yourself. The scale method can be used when you’re ready to run
longer tests with higher utilisation.

In [3]:
cluster.adapt(minimum=0, maximum=num_nodes)

In [4]:
# You may need to run this cell a few times while waiting for Dask to clean up
!squeue


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
               373   BigCats DSKS Jup andrewpe  R    1:48:56      1 mlerp-monash-node00
               380   BigCats dask-wor mhar0048  R       0:00      1 mlerp-monash-node00
               381   BigCats dask-wor mhar0048  R       0:00      1 mlerp-monash-node00
               382   BigCats dask-wor mhar0048  R       0:00      1 mlerp-monash-node00
               383   BigCats dask-wor mhar0048  R       0:00      1 mlerp-monash-node00
               375   BigCats DSKS Jup mhar0048  R    1:15:17      1 mlerp-monash-node00

Dask has a UI that will let you see how the tasks are being computed.
You won’t be able to connect to this with your web browser but VSCode
and Jupyter have extensions for you to connect to it.

Use the loopback address: http://127.0.0.1:8787 (Adjust the port to the
one listed when you make the client if needed)

Now let’s define a dask array and perform some computation. Dask arrays
are parallelised across your workers nodes so they can be greater than
the size of one worker’s memory. Dask evaluates lazily, retuning
‘futures’ which record the tasks needed to be completed in the compute
graph. They can be computed later for its value.

Dask also has parallelised implementations of dataframes and collections
of objects (called bags). These are written to be as similar as possible
to familiar libraries like numpy, pandas and pyspark. You can read more
about [arrays](https://docs.dask.org/en/stable/array.html),
[dataframes](https://docs.dask.org/en/stable/dataframe.html) and
[bags](https://docs.dask.org/en/stable/bag.html) with Dask’s
documentation.

In [5]:
import dask.array as da
x = da.random.random((1000, 1000, 1000))
x  # Note how the value of the array hasn't been computed yet


You can check squeue while this is running to see the jobs dynamically
spinning up to perform the computation.

In [8]:
x[0][0][:10].compute()


array([0.15885921, 0.35993257, 0.08629273, 0.19472071, 0.76723019,
       0.65067334, 0.33802127, 0.61253251, 0.37531874, 0.06273974])

Finally, we can shut down the SLURMCluster now that we’re done with it.

In [9]:
# Shut down the cluster
client.shutdown()


2023-08-21 02:44:02,335 - distributed.deploy.adaptive_core - INFO - Adaptive stop